<a href="https://colab.research.google.com/github/nkcong206/Travel-Recommendation-System/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Set up

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from sentence_transformers import SentenceTransformer
from langchain_google_genai import ChatGoogleGenerativeAI
import re
import json
import subprocess
import time
import threading
import psycopg2
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

e:\NhiAri\Code\GroupProject2024\groupproject2024\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
import getpass
import os

postgres_url = getpass.getpass("Enter your postgresql url: ")

In [4]:
model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')

### Gemini

In [ ]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [6]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

### Query

Có 2 thành phần chính filter và recommen
Filter là các câu query vào SQL. Cách tính giá sao cho phù hợp hay thế nào nằm ở phần Filter, Recommen có công dụng là đề xuất cái tốt nhất

Prompt để trích xuất thông tin. Để đưa được vào câu prompt thì trước hết phải lấy ra các giá trị xuất hiện bên trong cột style, amenities. Dưới đây là mẫu.


#### Get features

##### Hotels

In [7]:
# amenities
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

cur.execute("SET search_path TO travel_database, public;")

cur.execute("""
    SELECT DISTINCT unnest(amenities) AS unique_amenities
    FROM hotel;
""")

rows = cur.fetchall()

cur.close()
conn.close()

amenities_list = [row[0] for row in rows]
amenities_list_str = "\n    ".join(f'"{amenities_type}"' for amenities_type in amenities_list)
print(amenities_list_str)

"WiFi tại khu vực chung"
    "Money changer"
    "Bathtub"
    "Tiện nghi cho trẻ"
    "Quầy bar bên hồ bơi"
    "Heater"
    "Dù (ô) che nắng"
    "Đưa đón đến khu trượt tuyết (thu phí)"
    "Tiện nghi hội họp"
    "Sân quần vợt ngoài trời"
    "Trung tâm chăm sóc trẻ em"
    "Roll-in shower"
    "Vegetarian meal"
    "Hồ bơi"
    "Tủ lạnh (dùng chung)"
    "Máy photocopy"
    "Giữ trẻ"
    "Porter"
    "Bicycle storage"
    "Dịch vụ cho thuê xe đạp"
    "Conference room"
    "Dịch vụ trông trẻ có người giám hộ"
    "Grocery"
    "Giặt ủi"
    "Dịch vụ phòng 24 giờ"
    "Lò vi sóng"
    "Express check-out"
    "Đưa đón sân bay"
    "AC"
    "Dịch vụ phòng (có giới hạn thời gian)"
    "Bóng quần"
    "TV lounge"
    "A la carte lunch"
    "Bồn tắm nước nóng"
    "Gói cầu hôn lãng mạn"
    "Safety deposit box"
    "Vườn thú bán hoang dã"
    "Hồ bơi trẻ em"
    "Ghế dài tắm nắng"
    "Đưa đón đến trạm xe buýt (thu phí)"
    "Wifi (miễn phí)"
    "Thẩm mỹ viện"
    "Bữa trưa món tự chọn"

In [8]:
# style
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

cur.execute("SET search_path TO travel_database, public;")

cur.execute("""
    SELECT DISTINCT style
    FROM hotel
    WHERE style IS NOT NULL;
""")

rows = cur.fetchall()

cur.close()
conn.close()
style_list = [row[0] for row in rows]
style_list_str = "\n    ".join(f'"{style}"' for style in style_list)
print(style_list_str)

"Business
"
    "Adventure 
"
    "Romantic 
"
    "Eco-friendly 
"
    "Business 
"
    "Wellness 
"
    "Family-friendly 
"
    "Cultural
"
    "Romantic
"
    "Beachfront 
"
    "Luxury
"
    "Boutique
"
    "Eco-friendly
"
    "Adventure
"
    "Family-friendly
"
    "Boutique 
"
    "Cultural 
"
    "Wellness
"
    "Luxury 
"


##### Attractions

In [9]:
# types
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

cur.execute("SET search_path TO travel_database, public;")

cur.execute("""
    SELECT DISTINCT unnest(attraction_type) AS unique_attraction_type
    FROM touristattraction;
""")

rows = cur.fetchall()

cur.close()
conn.close()

att_type_list = [row[0] for row in rows]
att_type_list_str = "\n    ".join(f'"{att_type}"' for att_type in att_type_list)
print(att_type_list_str)

"Nhà hát và biểu diễn"
    "Viện bảo tàng lịch sử"
    "Thủy cung"
    "Khu vực đi dạo tham quan di tích lịch sử"
    "Trường đại học và trường học"
    "Quán bar và câu lạc bộ"
    "Khu vực đi dạo ngắm cảnh"
    "Viện bảo tàng nghệ thuật"
    "Vườn"
    "Di tích cổ"
    "Nhà thờ và nhà thờ lớn"
    "ATV và xe địa hình"
    "Đài kỷ niệm và tượng"
    "Cầu"
    "Xưởng vẽ và làm đồ gốm"
    "Chuyến tham quan văn hóa"
    "Núi"
    "Địa điểm giáo dục"
    "Khu liên hợp thể thao"
    "Buổi học và hội thảo"
    "Cửa hàng đồ cổ"
    "Sân gôn"
    "Triển lãm"
    "Đấu trường và sân vận động"
    "Phòng trưng bày nghệ thuật"
    "Điểm thu hút khách tham quan và thắng cảnh"
    "Địa điểm tâm linh"
    "Chợ hoa"
    "Cửa hàng của nhà máy"
    "Trung tâm nghệ thuật"
    "Quán bar rượu vang"
    "Căn cứ và doanh trại quân đội"
    "Địa điểm lịch sử"
    "Công viên nước"
    "Chuyến tham quan cà phê và trà"
    "Nhà hát"
    "Trung tâm trò chơi và giải trí"
    "Bảo t

##### Restaurant

In [10]:
# Type
# Establish the connection
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Set the search path to use the correct schema
cur.execute("SET search_path TO travel_database, public;")

# Query to extract distinct districts from the address composite type
cur.execute("""
    SELECT DISTINCT unnest(restaurant_type) AS unique_res_type
    FROM restaurant;
""")

# Fetch all rows
rows = cur.fetchall()

# Close the cursor and connection
cur.close()
conn.close()

# Convert the rows into a list and format the output
res_type_list = [row[0] for row in rows]
res_type_list_str = "\n    ".join(f'"{res_type}"' for res_type in res_type_list)
print(res_type_list_str)

"Karaoke"
    "Café/Dessert"
    "Buffet"
    "Ăn vặt/vỉa hè"
    "Tiệc cưới/Hội nghị"
    "Quán ăn"
    "Tiệm bánh"
    "Ăn chay"
    "Nhà hàng"


In [11]:
# Suitable for
# Establish the connection
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Set the search path to use the correct schema
cur.execute("SET search_path TO travel_database, public;")

# Query to extract distinct districts from the address composite type
cur.execute("""
    SELECT DISTINCT unnest(suitable_for) AS unique_res_suit
    FROM restaurant;
""")

# Fetch all rows
rows = cur.fetchall()

# Close the cursor and connection
cur.close()
conn.close()

# Convert the rows into a list and format the output
res_suit_list = [row[0] for row in rows]
res_suit_list_str = "\n    ".join(f'"{res_suit}"' for res_suit in res_suit_list)
print(res_suit_list_str)

"Uống bia - Nhậu"
    "Ăn gia đình"
    "Ăn chay"
    "Ăn Fastfood"
    "Đãi tiệc"
    "Tiếp khách"
    "Takeaway - Mang về"
    "Họp nhóm"
    "Ăn vặt"
    "Nghe nhạc"
    "Du lịch"
    "Ngắm cảnh"
    "Chụp hình - Quay phim"
    "BBQ - Món Nướng"
    "Tiệc ngoài trời"
    "Thư giãn"
    "Hẹn hò"
    "Buffet"


#### Prompt

Dựa vào câu prompt này để lấy ra các thông tin yêu cầu về hotel, restaurant, TouristAttraction

##### Extract first JSON

In [12]:
travel_type_list = ["Food Tour", "Văn hóa", "Thư giãn", "Trải nghiệm"]
companion_list = ["friends", "family", "colleagues"]
transport_list = ["self-drive car", "motorbike", "bicycle", "public transport"]
city_list = ["Hà Nội"]

In [13]:
template = """
You are an AI travel suggestion chatbot. Analyze the following travel request:

Request: "{travel_request}"

Extract general and specific requirements for Hotels, Restaurants, and Tourist Attractions, even if some are not explicitly mentioned. For each type, provide the following information:

**General Requirements:**
- Type: From this list: {travel_type_list} based on request or return null if not specified or only ask for one of Hotels, Restaurants, or Tourist Attractions.
- Number_of_people: Extract the number of people or return null if not specified.
- Companions: Extract the companions mentioned and from this list: {companion_list} or return null if not specified.
- Transportation: Identify the transportation method mentioned and from this list: {transport_list} or return null if not specified.
- Time: Any specific dates or time ranges mentioned or return null if not specified.
- City: The mentioned city (without "city" or "province") and from this list: {city_list}.
- Price_range: Specify as "low", "medium", or "high" based on the request.

**For Hotels, also identify:**
- Requirements: A summary text of specific requirements or preferences mentioned.
- Amenities: From this list: {amenities_list}
- Style: From this list: {style_list}

**For Restaurants, also identify:**
- Requirements: A summary text of specific requirements or preferences mentioned.
- Restaurant_Type: From this list: {restaurant_type_list}
- Suitable_For: From this list: {suitable_for_list}

**For Tourist Attractions, also identify:**
- Requirements: A list of specific requirements or preferences mentioned.
- Attraction_Type: From this list: {attraction_type_list}

Return the result using the following JSON format:

```json
{{
  "General": {{
    "Type": "...",
    "Number_of_people": "...",
    "Companion": "...",
    "Transportation": "...",
    "Time": "...",
    "City": "..."
    "Price_range": "...",
    "
  }},
  "Hotel": {{
    "Requirements": ...,
    "Amenities": [...],
    "Style": "..."
  }},
  "Restaurant": {{
    "Requirements": ...,
    "Restaurant_Type": "...",
    "Suitable_For": "..."
  }},
  "TouristAttraction": {{
    "Attraction_Type": "..."
  }}
}}

```

Ensure the JSON is valid. Use null for any unspecified information.
After the JSON output, add a note in Vietnamese:

"Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào, vui lòng cho tôi biết."
"""

In [14]:
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm

##### Ask again if there's missing infor

format json due to type

ưu tiên hỏi time

In [15]:
ask_template = """
You are an AI travel suggestion chatbot. Analyze the following travel request:

Request: "{travel_output_json}"

If any fields in the JSON are null, generate a question for the missing information. Only ask for fields that are **explicitly null**.

**Check the JSON output for any null values:**
- Type: From this list: {travel_type_list}
- Number_of_people: 
- Companions: From this list: {companion_list}
- Transportation: From this list: {transport_list}
- Time: 
- City: From this list: {city_list}
- Price_range: 

**For Hotels:**
- Requirements: 
- Amenities: From this list: {amenities_list}
- Style: From this list: {style_list}

**For Restaurants:**
- Requirements: 
- Restaurant_Type: From this list: {restaurant_type_list}
- Suitable_For: From this list: {suitable_for_list}

**For Tourist Attractions:**
- Requirements: 
- Attraction_Type: From this list: {attraction_type_list}

---

Analyze the JSON input. **If any field is null, generate the corresponding question**. **Only ask questions for fields that are explicitly null.** Ensure the final output is in **Vietnamese**, containing only the relevant questions in a natural, conversational format.

**Check the JSON output for any null values and generate appropriate questions:**

1. If `"Type"` is null, ask:  
   **"Bạn muốn tìm loại hình du lịch nào? (Ví dụ: Food Tour, Văn hóa, Thư giãn, hoặc Trải nghiệm)"**

2. If `"Number_of_people"` is null, ask:  
   **"Bạn đi bao nhiêu người? (Ví dụ: 1, 2, hoặc nhóm lớn hơn)"**

3. If `"Companion"` is null, ask:  
   **"Bạn đi cùng ai? (Bạn bè, Gia đình, hoặc Đồng nghiệp)"**

4. If `"Transportation"` is null, ask:  
   **"Bạn sẽ di chuyển bằng phương tiện gì? (Ví dụ: xe hơi tự lái, xe máy, hoặc phương tiện công cộng)"**

5. If `"Time"` is null, ask:  
   **"Bạn có kế hoạch đi vào thời gian nào không? (Ngày cụ thể hoặc khoảng thời gian)"**

6. If `"Price_range"` is null, ask:  
   **"Bạn muốn ngân sách cho chuyến đi này là bao nhiêu (thấp, trung bình, cao)?"**

---

### Example Output:
If the provided JSON input has `"Transportation"` and `"Time"` as `null`, the output will be:

```plaintext
Bạn sẽ di chuyển bằng phương tiện gì? (Ví dụ: xe hơi tự lái, xe máy, hoặc phương tiện công cộng)

Bạn có kế hoạch đi vào thời gian nào không? (Ngày cụ thể hoặc khoảng thời gian)

Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào, vui lòng cho tôi biết.
"""


In [16]:
ask_prompt = ChatPromptTemplate.from_template(ask_template)
ask_chain = ask_prompt | llm

##### Query from user requests

In [17]:
# Your input query
user_query = """
Gợi ý cho tôi một lộ trình du lịch tại Hà Nội với khách sạn sang trọng, có Bồn tắm, bể bơi và Mát-xa toàn thân, nhà hàng phục vụ món ăn truyền thống và một điểm tham quan nổi tiếng phù hợp cho trẻ em về đề tài lịch sử.
Tôi muốn biết thêm về các tiện nghi của khách sạn và phong cách của nhà hàng.
Chúng tôi đi 4 người.
"""

first response

In [18]:
def user_requires(chain, query, travel_type_list, companion_list, transport_list, city_list, 
                  amenities_list_str, style_list_str, res_type_list_str, res_suit_list_str, att_type_list_str):
    response = chain.invoke({
        "travel_request": query,
        "travel_type_list": travel_type_list,
        "companion_list": companion_list,
        "transport_list": transport_list,
        "city_list": city_list,
        "amenities_list": amenities_list_str,
        "style_list": style_list_str,
        "restaurant_type_list": res_type_list_str,
        "suitable_for_list": res_suit_list_str,
        "attraction_type_list": att_type_list_str
    })

    # Extract and parse the JSON response
    try:
        json_match = re.search(r'\{.*\}', response.content, re.DOTALL)
        if json_match:
            result_dict = json.loads(json_match.group(0))
            
            # Print the JSON result
            print("Extracted JSON Result:")
            print(json.dumps(result_dict, indent=2, ensure_ascii=False))
            return result_dict
        else:
            print("No JSON object found in the response.")
            return None
    except json.JSONDecodeError as e:
        print("Failed to decode JSON:", e)
        print("Raw response:", response.content)
        return None

In [20]:
user_requires_respond = user_requires(chain, user_query, travel_type_list, companion_list, transport_list, city_list, amenities_list_str, style_list_str, res_type_list_str, res_suit_list_str, att_type_list_str)

Extracted JSON Result:
{
  "General": {
    "Type": null,
    "Number_of_people": 4,
    "Companion": "family",
    "Transportation": null,
    "Time": null,
    "City": "Hà Nội",
    "Price_range": "high"
  },
  "Hotel": {
    "Requirements": "Khách sạn sang trọng, có bồn tắm, bể bơi và mát-xa toàn thân.",
    "Amenities": [
      "Bathtub",
      "Hồ bơi",
      "Mát-xa"
    ],
    "Style": "Luxury"
  },
  "Restaurant": {
    "Requirements": "Phục vụ món ăn truyền thống.",
    "Restaurant_Type": "Nhà hàng",
    "Suitable_For": "Ăn gia đình"
  },
  "TouristAttraction": {
    "Requirements": "Điểm tham quan nổi tiếng phù hợp cho trẻ em về đề tài lịch sử",
    "Attraction_Type": "Địa điểm lịch sử"
  }
}


ask again if missing values

In [21]:
def ask_user(ask_chain, response, travel_type_list, companion_list, transport_list, city_list, 
                  amenities_list_str, style_list_str, res_type_list_str, res_suit_list_str, att_type_list_str):
    response1 = ask_chain.invoke({
        "travel_output_json": response,
        "travel_type_list": travel_type_list,
        "companion_list": companion_list,
        "transport_list": transport_list,
        "city_list": city_list,
        "amenities_list": amenities_list_str,
        "style_list": style_list_str,
        "restaurant_type_list": res_type_list_str,
        "suitable_for_list": res_suit_list_str,
        "attraction_type_list": att_type_list_str
    })

    print(response1.content)

In [23]:
ask_again_respond = ask_user(ask_chain, user_requires_respond, travel_type_list, companion_list, transport_list, city_list, amenities_list_str, style_list_str, res_type_list_str, res_suit_list_str, att_type_list_str)
    

Bạn muốn tìm loại hình du lịch nào? (Ví dụ: Food Tour, Văn hóa, Thư giãn, hoặc Trải nghiệm)

Bạn có kế hoạch đi vào thời gian nào không? (Ngày cụ thể hoặc khoảng thời gian)

Bạn sẽ di chuyển bằng phương tiện gì? (Ví dụ: xe hơi tự lái, xe máy, hoặc phương tiện công cộng)

Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào, vui lòng cho tôi biết.



##### query from data

In [ ]:
def build_sql_query(table, requirements):
    conditions = []
    joins = ""
    
    # Xử lý điều kiện City
    if requirements.get("City"):
        conditions.append(f"city = '{requirements['City']}'")
    
    # Xử lý Price_range cho từng bảng
    if requirements.get("Price_range"):
        price_range = requirements["Price_range"]
        if table == "hotel":
            joins = "JOIN hotelprice ON hotel.hotel_id = hotelprice.hotel_id"
            if price_range == "low":
                conditions.append("hotelprice.price < 500000")
            elif price_range == "medium":
                conditions.append("hotelprice.price BETWEEN 500000 AND 2000000")
            elif price_range == "high":
                conditions.append("hotelprice.price > 2000000")
        
        elif table == "restaurant":
            if price_range == "low":
                conditions.append("CAST(restaurant.price_range->>'max' AS INTEGER) < 200000")
            elif price_range == "medium":
                conditions.append("CAST(restaurant.price_range->>'min' AS INTEGER) >= 200000 AND CAST(restaurant.price_range->>'max' AS INTEGER) <= 600000")
            elif price_range == "high":
                conditions.append("CAST(restaurant.price_range->>'min' AS INTEGER) > 600000")
        
        elif table == "touristattraction":
            joins = "JOIN attractionprice ON touristattraction.attraction_id = attractionprice.attraction_id"
            if price_range == "low":
                conditions.append("attractionprice.price < 500000")
            elif price_range == "medium":
                conditions.append("attractionprice.price BETWEEN 500000 AND 1500000")
            elif price_range == "high":
                conditions.append("attractionprice.price > 1500000")

    # Xử lý các điều kiện riêng cho từng bảng
    if table == "hotel":
        if requirements.get("Amenities"):
            amenities_condition = " AND ".join([f"'{amenity}' = ANY(amenities)" for amenity in requirements["Amenities"]])
            conditions.append(f"({amenities_condition})")
        if requirements.get("Style") and requirements["Style"] != "none":
            conditions.append(f"style = '{requirements['Style']}'")
    
    elif table == "restaurant":
        if requirements.get("Restaurant_Type"):
            conditions.append(f"'{requirements['Restaurant_Type']}' = ANY(restaurant_type)")
        if requirements.get("Suitable_For"):
            conditions.append(f"'{requirements['Suitable_For']}' = ANY(suitable_for)")
    
    elif table == "touristattraction":
        if requirements.get("Attraction_Type"):
            conditions.append(f"'{requirements['Attraction_Type']}' = ANY(attraction_type)")

    # Xây dựng câu lệnh WHERE và JOIN
    where_clause = " AND ".join(conditions)
    query = f"SELECT * FROM {table} {joins} WHERE {where_clause};" if conditions else f"SELECT * FROM {table};"
    
    return query


In [25]:
hotel_requirements = user_requires_respond.get("Hotel", {})
restaurant_requirements = user_requires_respond.get("Restaurant", {})
attraction_requirements = user_requires_respond.get("TouristAttraction", {})

# Build SQL queries
hotel_query = build_sql_query("hotel", hotel_requirements)
restaurant_query = build_sql_query("restaurant", restaurant_requirements)
attraction_query = build_sql_query("touristattraction", attraction_requirements)

print("Hotel Query:", hotel_query)
print("\nRestaurant Query:", restaurant_query)
print("\nAttraction Query:", attraction_query)

Hotel Query: SELECT * FROM hotel WHERE ('Bathtub' = ANY(amenities) AND 'Hồ bơi' = ANY(amenities) AND 'Mát-xa' = ANY(amenities)) AND style = 'Luxury';

Restaurant Query: SELECT * FROM restaurant WHERE 'Nhà hàng' = ANY(restaurant_type) AND 'Ăn gia đình' = ANY(suitable_for);

Attraction Query: SELECT * FROM touristattraction WHERE 'Địa điểm lịch sử' = ANY(attraction_type);


In [41]:
response = chain.invoke(
    {
        "travel_request": query,
        "amenities_list_str": amenities_list_str,
        "style_list_str": style_list_str,
        "res_type_list_str": res_type_list_str,
        "res_suit_list_str": res_suit_list_str,
        "att_type_list_str": att_type_list_str
    }
)

# Extract the JSON response content
response_gemini = str(response.content)

# Try to extract the JSON part from the response
try:
    # Use regex to extract the JSON object
    cleaned_json_str = re.search(r'\{.*\}', response_gemini, re.DOTALL).group(0)
    cleaned_json_str = cleaned_json_str.strip()

    # Load the JSON content as a dictionary
    result_dict = json.loads(cleaned_json_str)

    # Assign values to the three variables
    hotel_query = result_dict.get("Hotel", {})
    restaurant_query = result_dict.get("Restaurant", {})
    attraction_query = result_dict.get("TouristAttraction", {})

    # Print the results for verification
    print("Hotel Query:", json.dumps(hotel_query, indent=4, ensure_ascii=False))
    print("Restaurant Query:", json.dumps(restaurant_query, indent=4, ensure_ascii=False))
    print("Attraction Query:", json.dumps(attraction_query, indent=4, ensure_ascii=False))
except json.JSONDecodeError as e:
    print("Failed to decode JSON:", e)
    print("Raw response:", response_gemini)
except AttributeError:
    print("No valid JSON object found in the response.")

KeyError: 'Input to ChatPromptTemplate is missing variables {\'\\n        "Hotel"\'}.  Expected: [\'\\n        "Hotel"\', \'amenities_list_str\', \'att_type_list_str\', \'res_suit_list_str\', \'res_type_list_str\', \'style_list_str\', \'travel_request\'] Received: [\'travel_request\', \'amenities_list_str\', \'style_list_str\', \'res_type_list_str\', \'res_suit_list_str\', \'att_type_list_str\']\nNote: if you intended {\n        "Hotel"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\n        "Hotel"}}\'.'

In [ ]:
def get_embedding(text):
    """Trả về embedding của văn bản sử dụng mô hình SentenceTransformer."""
    embedding = model.encode(text)
    return embedding.tolist()

Câu truy xuất mẫu để lấy ra các hotel có Wifi trong amenities

### example

In [ ]:
# Kết nối với cơ sở dữ liệu PostgreSQL
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Lấy dữ liệu tiện nghi của các khách sạn có tên là "A"
cur.execute("""
    SELECT hotel_id, rating, amenities, style, description
    FROM hotel
    WHERE 'Wifi' = ANY(amenities);
""")

# Chuyển đổi dữ liệu thành DataFrame
data = cur.fetchall()
df = pd.DataFrame(data, columns=['hotel_id', 'rating','amenities', 'style', 'description'])

cur.close()

### Recommend

Bước recommen

Hotel

Yêu cầu dữ liệu của hotel cần trả về  sau khi query sql sẽ giống như sau (Có thể thay đổi như thêm khoảng cách, ...):

In [ ]:
data = [
    [1,1, 4.5, ['WiFi', 'Pool', 'Spa'], 'A beautiful luxury hotel with a spa and pool.', 90],
    [2,1, 4.5, ['WiFi', 'Pool', 'Spa'], 'A beautiful luxury hotel with a spa and pool.', 100],
    [3,2, 2.3, ['WiFi', 'Gym', 'Spa'], 'An eco-friendly hotel with gym facilities.', 80],
    [4,3, 1.2, ['WiFi', 'Beach'], 'A beachfront hotel with stunning ocean views.', 70]
]

df_data = pd.DataFrame(data, columns=['id', 'hotel_id', 'rating', 'amenities', 'description', 'price'])

query = [[  5, ['WiFi', 'Pool', 'Spa', 'Beach'], 'A beautiful luxury hotel.', 110]]

df_query = pd.DataFrame(query, columns=[ 'rating', 'amenities', 'description', 'price'])

df = pd.concat([df_query, df_data], ignore_index=True)

df_amenities = df[['id', 'amenities']].copy()
df_description = df[['id', 'description']].copy()
df_rating = df[['id', 'rating']].copy()
df_price = df[['id', 'price']].copy()

In [ ]:
mlb = MultiLabelBinarizer()
amenities_encoded = mlb.fit_transform(df_amenities['amenities'])

# Thay thế cột amenities bằng các vector one-hot encoded
df_amenities['amenities_nor'] = amenities_encoded.tolist()

similarities = [cosine_similarity([df_amenities['amenities_nor'][0]], [amenity_vector])[0][0]
                for amenity_vector in df_amenities['amenities_nor']]

# Thêm cột cosine similarity vào df_amenities
df_amenities['amenities_similarity'] = similarities

# Hiển thị kết quả
print(df_amenities)

    id                 amenities    amenities_nor  amenities_similarity
0  NaN  [WiFi, Pool, Spa, Beach]  [1, 0, 1, 1, 1]              1.000000
1  1.0         [WiFi, Pool, Spa]  [0, 0, 1, 1, 1]              0.866025
2  2.0         [WiFi, Pool, Spa]  [0, 0, 1, 1, 1]              0.866025
3  3.0          [WiFi, Gym, Spa]  [0, 1, 0, 1, 1]              0.577350
4  4.0             [WiFi, Beach]  [1, 0, 0, 0, 1]              0.707107


In [ ]:
df_description['description_embedding'] = df_description['description'].apply(get_embedding)
# Tính cosine similarity giữa mỗi embedding và embedding của hàng 0
similarities = [cosine_similarity([df_description['description_embedding'][0]], [embedding])[0][0]
                for embedding in df_description['description_embedding']]

# Thêm cột cosine similarity vào df_description
df_description['description_similarity'] = similarities

# Hiển thị kết quả
print(df_description)

    id                                    description  \
0  NaN                      A beautiful luxury hotel.   
1  1.0  A beautiful luxury hotel with a spa and pool.   
2  2.0  A beautiful luxury hotel with a spa and pool.   
3  3.0     An eco-friendly hotel with gym facilities.   
4  4.0  A beachfront hotel with stunning ocean views.   

                               description_embedding  description_similarity  
0  [0.09188824146986008, -0.21028681099414825, -0...                1.000000  
1  [0.09868957102298737, -0.14982357621192932, -0...                0.817774  
2  [0.09868957102298737, -0.14982357621192932, -0...                0.817774  
3  [0.29612961411476135, -0.10158709436655045, -0...                0.323857  
4  [-0.05280661582946777, -0.21619683504104614, 0...                0.215760  


In [ ]:
# scaler = MinMaxScaler()
# df_rating['rating_similarity'] = scaler.fit_transform(df_rating[['rating']])

max_value = df_rating['rating'].max()

df_rating['rating_similarity'] =  df_rating['rating']  / max_value

print(df_rating)

    id  rating  rating_similarity
0  NaN     5.0               1.00
1  1.0     4.5               0.90
2  2.0     4.5               0.90
3  3.0     2.3               0.46
4  4.0     1.2               0.24


In [ ]:
# scaler = MinMaxScaler()

# df_price['price_similarity'] = scaler.fit_transform(df_price[['price']])

max_value = df_price['price'].max()

df_price['price_similarity'] =  df_price['price']  / max_value

print(df_price)

    id  price  price_similarity
0  NaN    110          1.000000
1  1.0     90          0.818182
2  2.0    100          0.909091
3  3.0     80          0.727273
4  4.0     70          0.636364


In [ ]:
df_ranking = df[['id', 'hotel_id']].copy()
df_ranking = df_ranking.merge(df_amenities[['id', 'amenities_similarity']], on='id', how='left')
df_ranking = df_ranking.merge(df_description[['id', 'description_similarity']], on='id', how='left')
df_ranking = df_ranking.merge(df_rating[['id', 'rating_similarity']], on='id', how='left')
df_ranking = df_ranking.merge(df_price[['id', 'price_similarity']], on='id', how='left')
df_ranking['total_similarity'] = (df_ranking['amenities_similarity'] +
                                  df_ranking['description_similarity'] +
                                  df_ranking['rating_similarity'] -
                                  2*df_ranking['price_similarity'])
# Hiển thị kết quả
df_ranking = df_ranking.iloc[1:]
print(df_ranking)

    id  hotel_id  amenities_similarity  description_similarity  \
1  1.0       1.0              0.866025                0.817774   
2  2.0       1.0              0.866025                0.817774   
3  3.0       2.0              0.577350                0.323857   
4  4.0       3.0              0.707107                0.215760   

   rating_similarity  price_similarity  total_similarity  
1               0.90          0.818182          0.947436  
2               0.90          0.909091          0.765617  
3               0.46          0.727273         -0.093338  
4               0.24          0.636364         -0.109860  


In [ ]:
# Tìm giá trị lớn nhất trong cột 'total_similarity'
max_total_similarity_row = df_ranking.loc[df_ranking['total_similarity'].idxmax()]

max_total_similarity_id = max_total_similarity_row['id']
hotel_id = max_total_similarity_row['hotel_id']

print("ID: ", max_total_similarity_id)
print("Hotel ID: ", hotel_id)

ID:  1.0
Hotel ID:  1.0


TouristAttraction

In [ ]:
data = [
    [1, 1, 4.5, 'A historic site with rich cultural heritage and beautiful architecture.', 100],
    [2, 1, 4.5, 'A stunning natural attraction with breathtaking views and scenic trails.', 90],
    [3, 2, 2.3, 'An eco-friendly museum with interactive exhibits and educational displays.',80],
    [4, 3, 1.2, 'A popular beachfront attraction with stunning ocean views and activities.',60]
]

df_data = pd.DataFrame(data, columns=['id', 'attraction_id','rating', 'description', 'price'])

query = [[ 4.0, 'A new beach attraction with calm waters and amenities.', 110]]

df_query = pd.DataFrame(query, columns=['rating', 'description', 'price'])

df = pd.concat([df_query, df_data], ignore_index=True)

df_description = df[['id', 'description']].copy()
df_rating = df[['id', 'rating']].copy()
df_price = df[['id', 'price']].copy()

In [ ]:
# scaler = MinMaxScaler()

# df_price['price_similarity'] = scaler.fit_transform(df_price[['price']])

max_value = df_price['price'].max()

df_price['price_similarity'] =  df_price['price']  / max_value

print(df_price)

    id  price  price_similarity
0  NaN    110          1.000000
1  1.0    100          0.909091
2  2.0     90          0.818182
3  3.0     80          0.727273
4  4.0     60          0.545455


In [ ]:
# scaler = MinMaxScaler()
# df_rating['rating_similarity'] = scaler.fit_transform(df_rating[['rating']])

max_value = df_rating['rating'].max()

df_rating['rating_similarity'] =  df_rating['rating']  / max_value

print(df_rating)

    id  rating  rating_similarity
0  NaN     4.0           0.888889
1  1.0     4.5           1.000000
2  2.0     4.5           1.000000
3  3.0     2.3           0.511111
4  4.0     1.2           0.266667


In [ ]:
df_description['description_embedding'] = df_description['description'].apply(get_embedding)
# Tính cosine similarity giữa mỗi embedding và embedding của hàng 0
similarities = [cosine_similarity([df_description['description_embedding'][0]], [embedding])[0][0]
                for embedding in df_description['description_embedding']]

# Thêm cột cosine similarity vào df_description
df_description['description_similarity'] = similarities

# Hiển thị kết quả
print(df_description)

    id                                        description  \
0  NaN  A new beach attraction with calm waters and am...   
1  1.0  A historic site with rich cultural heritage an...   
2  2.0  A stunning natural attraction with breathtakin...   
3  3.0  An eco-friendly museum with interactive exhibi...   
4  4.0  A popular beachfront attraction with stunning ...   

                               description_embedding  description_similarity  
0  [0.10848864167928696, -0.5871910452842712, 0.1...                1.000000  
1  [0.23401594161987305, -0.004697909113019705, -...                0.123529  
2  [-0.08091728389263153, -0.4650380313396454, 0....                0.429118  
3  [0.016766875982284546, -0.03275265917181969, 0...                0.098162  
4  [0.08534727990627289, -0.5192856788635254, -0....                0.524035  


In [ ]:
df_ranking = df[['id', 'attraction_id']].copy()
df_ranking = df_ranking.merge(df_description[['id', 'description_similarity']], on='id', how='left')
df_ranking = df_ranking.merge(df_rating[['id', 'rating_similarity']], on='id', how='left')
df_ranking = df_ranking.merge(df_price[['id', 'price_similarity']], on='id', how='left')
df_ranking['total_similarity'] = (df_ranking['description_similarity'] +
                                  df_ranking['rating_similarity'] -
                                  2*df_ranking['price_similarity'])
# Hiển thị kết quả
df_ranking = df_ranking.iloc[1:]
print(df_ranking)

    id  attraction_id  description_similarity  rating_similarity  \
1  1.0            1.0                0.123529           1.000000   
2  2.0            1.0                0.429118           1.000000   
3  3.0            2.0                0.098162           0.511111   
4  4.0            3.0                0.524035           0.266667   

   price_similarity  total_similarity  
1          0.909091         -0.694653  
2          0.818182         -0.207245  
3          0.727273         -0.845272  
4          0.545455         -0.300207  


In [ ]:
max_total_similarity_row = df_ranking.loc[df_ranking['total_similarity'].idxmax()]

max_total_similarity_id = max_total_similarity_row['id']
attraction_id = max_total_similarity_row['attraction_id']

print("ID: ", max_total_similarity_id)
print("Attraction ID: ", attraction_id)

ID:  2.0
Attraction ID:  1.0


Restaurant

In [ ]:
data = [
    [1, 1, ['Fast Food', 'Casual Dining'], ['Family', 'Groups'], 4.2, 'A popular fast food restaurant with a variety of burgers and fries.'],
    [2, 2, ['Fine Dining'], ['Couples', 'Groups'], 4.8, 'An upscale restaurant offering gourmet dishes and a sophisticated ambiance.'],
    [3, 3, ['Casual Dining'], ['Family', 'Couples'], 3.9, 'A relaxed dining place with a wide menu catering to various tastes.'],
    [4, 4, ['Buffet'], ['Groups'], 4.0, 'A buffet restaurant with an extensive selection of dishes and a family-friendly environment.']
]

df_data = pd.DataFrame(data, columns=['id', 'restaurant_id', 'restaurant_type', 'suitable_for', 'rating', 'description'])

query = [[0, ['Casual Dining', 'Buffet'], ['Family', 'Groups'], 4.1, 'A casual dining spot with a diverse menu and a friendly atmosphere.']]
df_query = pd.DataFrame(query, columns=[ 'id','restaurant_type', 'suitable_for', 'rating', 'description'])

df = pd.concat([df_query, df_data], ignore_index=True)

df_description = df[['id', 'description']].copy()
df_rating = df[['id', 'rating']].copy()
df_type = df[['id', 'restaurant_type']].copy()
df_suitable = df[['id', 'suitable_for']].copy()

In [ ]:
df_description['description_embedding'] = df_description['description'].apply(get_embedding)
# Tính cosine similarity giữa mỗi embedding và embedding của hàng 0
similarities = [cosine_similarity([df_description['description_embedding'][0]], [embedding])[0][0]
                for embedding in df_description['description_embedding']]

# Thêm cột cosine similarity vào df_description
df_description['description_similarity'] = similarities

# Hiển thị kết quả
print(df_description)

   id                                        description  \
0   0  A casual dining spot with a diverse menu and a...   
1   1  A popular fast food restaurant with a variety ...   
2   2  An upscale restaurant offering gourmet dishes ...   
3   3  A relaxed dining place with a wide menu cateri...   
4   4  A buffet restaurant with an extensive selectio...   

                               description_embedding  description_similarity  
0  [0.03374265506863594, -0.18019609153270721, 0....                1.000000  
1  [0.07862076163291931, 0.08941935002803802, -0....                0.507882  
2  [-0.0810798928141594, -0.05817342549562454, 0....                0.358109  
3  [-0.061645783483982086, -0.08416043967008591, ...                0.652577  
4  [0.1955445110797882, -0.04515630751848221, 0.2...                0.590592  


In [ ]:
# scaler = MinMaxScaler()
# df_rating['rating_similarity'] = scaler.fit_transform(df_rating[['rating']])

max_value = df_rating['rating'].max()

df_rating['rating_similarity'] =  df_rating['rating']  / max_value

print(df_rating)

   id  rating  rating_similarity
0   0     4.1           0.854167
1   1     4.2           0.875000
2   2     4.8           1.000000
3   3     3.9           0.812500
4   4     4.0           0.833333


In [ ]:
mlb = MultiLabelBinarizer()
suitable_encoded = mlb.fit_transform(df_suitable['suitable_for'])

df_suitable['suitable_nor'] = suitable_encoded.tolist()

similarities = [cosine_similarity([df_suitable['suitable_nor'][0]], [suitable_vector])[0][0]
                for suitable_vector in df_suitable['suitable_nor']]

df_suitable['suitable_similarity'] = similarities

print(df_suitable)

   id       suitable_for suitable_nor  suitable_similarity
0   0   [Family, Groups]    [0, 1, 1]             1.000000
1   1   [Family, Groups]    [0, 1, 1]             1.000000
2   2  [Couples, Groups]    [1, 0, 1]             0.500000
3   3  [Family, Couples]    [1, 1, 0]             0.500000
4   4           [Groups]    [0, 0, 1]             0.707107


In [ ]:
mlb = MultiLabelBinarizer()
type_encoded = mlb.fit_transform(df_type['restaurant_type'])

df_type['type_nor'] = type_encoded.tolist()

similarities = [cosine_similarity([df_type['type_nor'][0]], [type_vector])[0][0]
                for type_vector in df_type['type_nor']]

df_type['type_similarity'] = similarities

print(df_type)

   id             restaurant_type      type_nor  type_similarity
0   0     [Casual Dining, Buffet]  [1, 1, 0, 0]         1.000000
1   1  [Fast Food, Casual Dining]  [0, 1, 1, 0]         0.500000
2   2               [Fine Dining]  [0, 0, 0, 1]         0.000000
3   3             [Casual Dining]  [0, 1, 0, 0]         0.707107
4   4                    [Buffet]  [1, 0, 0, 0]         0.707107


In [ ]:
df_ranking = df[['id', 'restaurant_id']].copy()
df_ranking = df_ranking.merge(df_description[['id', 'description_similarity']], on='id', how='left')
df_ranking = df_ranking.merge(df_rating[['id', 'rating_similarity']], on='id', how='left')
df_ranking = df_ranking.merge(df_suitable[['id', 'suitable_similarity']], on='id', how='left')
df_ranking = df_ranking.merge(df_type[['id', 'type_similarity']], on='id', how='left')
df_ranking['total_similarity'] = (df_ranking['description_similarity'] +
                                  df_ranking['rating_similarity'] +
                                  df_ranking['suitable_similarity'] +
                                  df_ranking['type_similarity']
                                  )
# Hiển thị kết quả
df_ranking = df_ranking.iloc[1:]
print(df_ranking)

   id  restaurant_id  description_similarity  rating_similarity  \
1   1            1.0                0.507882           0.875000   
2   2            2.0                0.358109           1.000000   
3   3            3.0                0.652577           0.812500   
4   4            4.0                0.590592           0.833333   

   suitable_similarity  type_similarity  total_similarity  
1             1.000000         0.500000          2.882882  
2             0.500000         0.000000          1.858109  
3             0.500000         0.707107          2.672184  
4             0.707107         0.707107          2.838139  


In [ ]:
max_total_similarity_row = df_ranking.loc[df_ranking['total_similarity'].idxmax()]

max_total_similarity_id = max_total_similarity_row['id']
restaurant_id = max_total_similarity_row['restaurant_id']

print("ID: ", max_total_similarity_id)
print("Restaurant ID: ", restaurant_id)

ID:  1.0
Restaurant ID:  1.0


Cũ

In [ ]:
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Đặt schema hiện tại
cur.execute("SET search_path TO travel_database, public;")

# Tạo vector cho mô tả truy vấn


# Thành phố cần tìm
response = chain.invoke(query)
response_gemini = str(response.content)
cleaned_json_str = re.search(r'\{.*?\}', response_gemini, re.DOTALL).group(0)
result_dict = json.loads(cleaned_json_str)


embedding_query = model.encode(result_dict['Description'])
embedding_query_list = embedding_query.tolist()

# Kiểm tra xem bảng Hotel có tồn tại không
cur.execute("SELECT * FROM information_schema.tables WHERE table_name = 'hotel';")
if not cur.fetchone():
    print("Bảng 'Hotel' không tồn tại trong schema 'travel_database'.")
else:
    # Thực hiện truy vấn để tìm 2 khách sạn có độ tương đồng cao nhất
    cur.execute("""
        WITH query_embedding AS (
            SELECT %s::vector(768) AS query_vector
        ),
        similarity_scores AS (
            SELECT
                h.hotel_id,
                h.name,
                h.address,
                h.rating,
                h.description,
                (1 - (h.embedding_description <=> query_vector)) AS similarity
            FROM
                Hotel h
                CROSS JOIN query_embedding
            WHERE
                (h.address).city = %s
            ORDER BY
                similarity DESC
            LIMIT 2
        )
        -- Chọn 2 khách sạn có độ tương đồng cao nhất
        SELECT
            hotel_id,
            name,
            address,
            rating,
            description,
            similarity
        FROM
            similarity_scores;
    """, (embedding_query_list, result_dict['City']))

    # Lấy kết quả và in ra
    rows = cur.fetchall()

    print(f"Yêu cầu người dùng: {query}")
    for row in rows:
        print(f"HotelID: {row[0]}")
        print(f"Name: {row[1]}")
        print(f"Address: {row[2]}")
        print(f"Rating: {row[3]}")
        print(f"Description: {row[4]}")
        print(f"Similarity: {row[5]}")
        print("-" * 40)

    print(result_dict)
# Đóng kết nối
cur.close()
conn.close()

Yêu cầu người dùng: 
Gợi ý cho tôi 1 khách sạn ở Cầu Giấy, Hà Nội với chất lượng dịch vụ cao cấp!

HotelID: 1
Name: Khách sạn A
Address: (0,"Cầu Giấy","Hà Nội")
Rating: 4.5
Description: Khách sạn cao cấp gần trung tâm Hà Nội, với các tiện nghi hiện đại và dịch vụ khách hàng xuất sắc.
Similarity: 0.23988989521233273
----------------------------------------
HotelID: 4
Name: Khách sạn D
Address: (0,"Cầu Giấy","Hà Nội")
Rating: 4.9
Description: Khách sạn sang trọng với dịch vụ 5 sao, bao gồm spa, bể bơi và nhà hàng cao cấp.
Similarity: 0.19820067049122325
----------------------------------------
{'Type': 'Hotel', 'District': 'Cầu Giấy', 'City': 'Hà Nội', 'Number_of_people': None, 'Price': None, 'Rating': None, 'Description': 'chất lượng dịch vụ cao cấp'}
